In [1]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px


In [2]:

# Conectando ao banco
conn = sqlite3.connect(r'/content/vendas_marketing.db')

df_vendas = pd.read_sql('SELECT * FROM Vendas', conn)
df_produtos = pd.read_sql('SELECT * FROM Produtos', conn)
df_campanhas_marketing = pd.read_sql('SELECT * FROM Campanhas_Marketing', conn)
df_clientes = pd.read_sql('SELECT * FROM Clientes', conn)
df_interacoes_marketing = pd.read_sql('SELECT * FROM Interacoes_Marketing', conn)




# A. Análise de Vendas





1. Total de Vendas por Canal:
Calcule o valor total de vendas por canal de aquisição (Outbound vs Inbound) em um período específico (por exemplo, no último trimestre).

In [3]:
valor_por_canal = df_vendas['canal_aquisicao'].value_counts()
valor_por_canal


,count
canal_aquisicao,
Outbound,493
Inbound,492


In [4]:
fig = px.pie(
    valor_por_canal,
    names=valor_por_canal.index,
    values=valor_por_canal.values,
    title='Valor Total por Canal de Aquisição'
)
fig.update_traces(textinfo='percent+label')
fig.show()



---



2. Top Produtos:
Identifique os 5 produtos com maior volume de vendas e calcule a margem de lucro média para esses produtos.



In [5]:
# Relacionando as tabelas Vendas com Produtos
vendas_produtos = df_vendas.join(df_produtos.set_index('id_produto'), on='id_produto', how='left')

# Alterando para datetime a coluna data_venda
vendas_produtos['data_venda'] = pd.to_datetime(vendas_produtos['data_venda'])

In [6]:
# Definindo a coluna id_vendas como index
vendas_produtos.set_index('id_venda', inplace=True)

In [7]:
# Verificando as infos das tabelas
vendas_produtos.head(5)

,id_cliente,id_produto,id_campanha,data_venda,quantidade,valor_total,canal_aquisicao,nome_produto,categoria,preco_unitario,custo_unitario
id_venda,,,,,,,,,,,
1,273,13,8.0,2025-06-07,4,2623.20,Inbound,GS Engage - Hic,SaaS,655.80,537.76
2,4044,21,11.0,2025-07-27,10,7295.70,Outbound,High Growth - Nisi,Consultoria,729.57,437.74
3,714,19,13.0,2025-10-29,1,142.35,Outbound,Station AI - Blanditiis,SaaS,142.35,106.76
4,1394,8,7.0,2025-12-01,1,871.78,Outbound,GS Engage - Commodi,SaaS,871.78,531.79
5,2644,8,10.0,2025-01-16,1,871.78,Outbound,GS Engage - Commodi,SaaS,871.78,531.79


In [8]:
# Inserindo a coluna lucro total
vendas_produtos['lucro_total'] = (vendas_produtos['preco_unitario'] - vendas_produtos['custo_unitario']) * vendas_produtos['quantidade']

In [9]:
#Validando as informações
vendas_produtos

,id_cliente,id_produto,id_campanha,data_venda,quantidade,valor_total,canal_aquisicao,nome_produto,categoria,preco_unitario,custo_unitario,lucro_total
id_venda,,,,,,,,,,,,
1,273,13,8.0,2025-06-07,4,2623.20,Inbound,GS Engage - Hic,SaaS,655.80,537.76,472.16
2,4044,21,11.0,2025-07-27,10,7295.70,Outbound,High Growth - Nisi,Consultoria,729.57,437.74,2918.30
3,714,19,13.0,2025-10-29,1,142.35,Outbound,Station AI - Blanditiis,SaaS,142.35,106.76,35.59
4,1394,8,7.0,2025-12-01,1,871.78,Outbound,GS Engage - Commodi,SaaS,871.78,531.79,339.99
5,2644,8,10.0,2025-01-16,1,871.78,Outbound,GS Engage - Commodi,SaaS,871.78,531.79,339.99
...,...,...,...,...,...,...,...,...,...,...,...,...
981,416,5,NaN,2025-11-20,5,77.35,Inbound,Station AI - Similique,SaaS,15.47,9.59,29.40
982,4005,11,NaN,2025-04-19,10,2324.10,Inbound,Demanda Infinita - Maxime,Treinamento,232.41,204.52,278.90
983,4061,9,NaN,2025-10-09,10,401.90,Outbound,GS Engage - Doloremque,SaaS,40.19,29.34,108.50


In [10]:
vendas_produtos['margem_lucro'] = (vendas_produtos['lucro_total'] / vendas_produtos['valor_total']) * 100

In [11]:
vendas_produtos

,id_cliente,id_produto,id_campanha,data_venda,quantidade,valor_total,canal_aquisicao,nome_produto,categoria,preco_unitario,custo_unitario,lucro_total,margem_lucro
id_venda,,,,,,,,,,,,,
1,273,13,8.0,2025-06-07,4,2623.20,Inbound,GS Engage - Hic,SaaS,655.80,537.76,472.16,17.999390
2,4044,21,11.0,2025-07-27,10,7295.70,Outbound,High Growth - Nisi,Consultoria,729.57,437.74,2918.30,40.000274
3,714,19,13.0,2025-10-29,1,142.35,Outbound,Station AI - Blanditiis,SaaS,142.35,106.76,35.59,25.001756
4,1394,8,7.0,2025-12-01,1,871.78,Outbound,GS Engage - Commodi,SaaS,871.78,531.79,339.99,38.999518
5,2644,8,10.0,2025-01-16,1,871.78,Outbound,GS Engage - Commodi,SaaS,871.78,531.79,339.99,38.999518
...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,416,5,NaN,2025-11-20,5,77.35,Inbound,Station AI - Similique,SaaS,15.47,9.59,29.40,38.009050
982,4005,11,NaN,2025-04-19,10,2324.10,Inbound,Demanda Infinita - Maxime,Treinamento,232.41,204.52,278.90,12.000344
983,4061,9,NaN,2025-10-09,10,401.90,Outbound,GS Engage - Doloremque,SaaS,40.19,29.34,108.50,26.996765


In [12]:
#Analisando os valores nulos na coluna id_campanha. Possivelmente vendas sem a utilização de campanhas.
vendas_produtos[vendas_produtos['id_campanha'].isnull()].head(5)

,id_cliente,id_produto,id_campanha,data_venda,quantidade,valor_total,canal_aquisicao,nome_produto,categoria,preco_unitario,custo_unitario,lucro_total,margem_lucro
id_venda,,,,,,,,,,,,,
658,767,6,NaN,2025-11-18,1,306.68,Outbound,High Growth - Quis,Consultoria,306.68,239.21,67.47,22.000130
659,725,18,NaN,2025-08-02,1,732.90,Outbound,High Growth - Quis,Consultoria,732.90,410.42,322.48,44.000546
660,4624,13,NaN,2025-08-17,3,1967.40,Inbound,GS Engage - Hic,SaaS,655.80,537.76,354.12,17.999390
661,860,11,NaN,2025-03-15,1,232.41,Outbound,Demanda Infinita - Maxime,Treinamento,232.41,204.52,27.89,12.000344
662,4016,9,NaN,2025-02-21,7,281.33,Inbound,GS Engage - Doloremque,SaaS,40.19,29.34,75.95,26.996765


In [13]:

# Selectionando os tops 5 produtos
top5_produtos = vendas_produtos['nome_produto'].value_counts().head(5).index

# Filtrando as vendas dos top 5 produtos
vendas_top5 = vendas_produtos[vendas_produtos['nome_produto'].isin(top5_produtos)]

# Agrupando e calcula métricas adicionais
resumo_top5 = vendas_top5.groupby('nome_produto')['margem_lucro'].mean().sort_values(ascending=False).round(1)
resumo_top5



,margem_lucro
nome_produto,
Station AI - Harum,48.0
High Growth - Nisi,40.0
High Growth - Quis,28.0
GS Engage - Doloremque,27.0
Imersão - Aliquam,17.0


In [14]:

fig = px.bar(resumo_top5, title='Lucro médio por Produto')
fig.update_layout(xaxis_title='Produto', yaxis_title='Lucro médio')
fig.update_legends(title_text='Legenda:')
fig.update_traces(texttemplate='%{y}%')
fig.show()






---



3. Segmentação de Clientes:
Compare o ticket médio entre os clientes dos segmentos B2B e B2C.

In [15]:
vendas_clientes = df_vendas.join(df_clientes.set_index('id_cliente'), on='id_cliente', how='left')
vendas_clientes.set_index('id_venda', inplace=True)
# Convert 'data_venda' to datetime after the join
vendas_clientes['data_venda'] = pd.to_datetime(vendas_clientes['data_venda'])

In [16]:
vendas_clientes

,id_cliente,id_produto,id_campanha,data_venda,quantidade,valor_total,canal_aquisicao,nome,cidade,segmento
id_venda,,,,,,,,,,
1,273,13,8.0,2025-06-07,4,2623.20,Inbound,Arthur Ramos,Cavalcanti,B2C
2,4044,21,11.0,2025-07-27,10,7295.70,Outbound,Dra. Heloísa Montenegro,Mendes,B2B
3,714,19,13.0,2025-10-29,1,142.35,Outbound,João Guilherme Porto,Cardoso,B2C
4,1394,8,7.0,2025-12-01,1,871.78,Outbound,Srta. Caroline Rodrigues,Azevedo de Moraes,B2C
5,2644,8,10.0,2025-01-16,1,871.78,Outbound,Ravi Porto,Barros do Campo,B2C
...,...,...,...,...,...,...,...,...,...,...
981,416,5,NaN,2025-11-20,5,77.35,Inbound,Maria Fernanda Fonseca,Lima das Pedras,B2B
982,4005,11,NaN,2025-04-19,10,2324.10,Inbound,Dra. Vitória Macedo,Andrade,B2C
983,4061,9,NaN,2025-10-09,10,401.90,Outbound,Dr. Arthur Gabriel Ferreira,Montenegro Grande,B2B


In [17]:
ticket_por_cliente = (
    vendas_clientes
        .groupby('segmento')
        .agg(
            faturamento_total=('valor_total', 'sum'),
            numero_vendas=('valor_total', 'count')
        )
        .assign(ticket_medio=lambda x: (x['faturamento_total'] / x['numero_vendas']).round(2))
        .reset_index()
        .sort_values('ticket_medio', ascending=False)
)


In [18]:
ticket_por_cliente

,segmento,faturamento_total,numero_vendas,ticket_medio
0,B2B,1215913.12,491,2476.40
1,B2C,1215108.93,494,2459.73


In [19]:
fig = px.box(ticket_por_cliente, x='segmento', y='ticket_medio', title='Ticket médio por segmento de cliente')
fig.update_layout(xaxis_title='Segmento', yaxis_title='Ticket médio')
fig.show()




---



4. Sazonalidade:
Analise o padrão de vendas ao longo do ano, identificando picos e quedas, e proponha possíveis explicações baseadas no comportamento de compra.

In [20]:
# Definindo a coluna data_venda para o tipo datetime
df_vendas['data_venda'] =  pd.to_datetime(df_vendas['data_venda'])

In [21]:
# Agrupar por mês e somar o valor total
vendas_mensais = df_vendas.groupby(df_vendas['data_venda'].dt.to_period('M'))['valor_total'].sum().reset_index()

# Converter de Period para datetime para usar no gráfico
vendas_mensais['data_venda'] = vendas_mensais['data_venda'].dt.to_timestamp()

fig = px.line(
    vendas_mensais,
    x='data_venda',
    y='valor_total',
    markers=True,
    title='Padrão de Vendas ao Longo do Ano'
)

fig.update_layout(
    xaxis_title='Mês',
    yaxis_title='Valor Total de Vendas'
)

fig.show()


In [22]:
# Agrupando por mês e segmento
vendas_por_segmento = (
    vendas_clientes
    .groupby([vendas_clientes['data_venda'].dt.to_period('M'), 'segmento'])['valor_total']
    .sum()
    .reset_index()
)

# Alterando de Period para datetime
vendas_por_segmento['data_venda'] = vendas_por_segmento['data_venda'].dt.to_timestamp()


fig = px.line(
    vendas_por_segmento,
    x='data_venda',
    y='valor_total',
    color='segmento',
    markers=True,
    title='Vendas por Segmento ao Longo do Ano'
)

fig.update_layout(
    xaxis_title='Mês',
    yaxis_title='Valor Total de Vendas'
)

fig.show()


Jan - No inicio do ano até fevereiro é possível notar uma queda, possívelmente devido as festas ocorridas no final do ano.

Fev | Abr - Ocorre um aumento devido ao retorno da economia após as vendas do final ano. Durante esses meses ocorrem eventos promocionais e data comemorativas: Páscoa e carnaval.

Abr | Maio - Há uma queda das vendas ocorrido, possívelmente, pela eventos promocionais para a data comemorativa de dias das mães.

Mai | Jul - Aumento de vendas ocorrida devido as promoções de meio de ano. E também datas comemorativas como dias dos namorados e festas juninas.

Ago - Queda devido ao mês de Julho, muitas pessoas planejam viagens em familia.

Set | Out - volta a subir  pode estar ligado a campanhas de primavera ou volta às aulas.

No mês de novembro ocorre uma "anormalidade" nas vendas, pois o padrão é as vendas aumentarem. interessante, porque em muitos setores dezembro é pico (Natal). Aqui, pode indicar que seu negócio não é fortemente atrelado ao varejo de fim de ano












# B. Análise de Marketing

5. Eficiência das Campanhas:
Determine quais campanhas tiveram maior taxa de conversão, relacionando o número de interações do tipo "Conversão" com o orçamento e custo das campanhas.

In [48]:
# Relacionando as tabelas
campanhas_marketing = df_campanhas_marketing.join(df_interacoes_marketing.set_index('id_campanha'), on='id_campanha', how='left')

# Removendo o nome campanha de cada célula, mnatendo somente da coluna
df_campanhas_marketing['nome_campanha'] = df_campanhas_marketing['nome_campanha'].str.replace('Campanha', '')


In [50]:
campanhas_marketing.to_excel('campanhas_marketing.xlsx', index=False)

In [49]:
campanhas_marketing

,id_campanha,nome_campanha,canal_marketing,data_inicio,data_fim,orcamento,custo,id_interacao,id_cliente,data_interacao,tipo_interacao
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,14,3189,2025-06-19,Visualização
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,51,1758,2025-04-21,Visualização
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,64,206,2025-01-21,Visualização
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,72,4202,2025-05-24,Visualização
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,80,4915,2025-07-09,Conversão
...,...,...,...,...,...,...,...,...,...,...,...
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,14811,2732,2025-05-19,Conversão
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,14842,1990,2025-09-25,Visualização
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,14892,4426,2025-03-05,Clique
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,14936,2488,2025-08-15,Visualização


In [25]:
taxa_conversao = (
    campanhas_marketing
    .groupby(['nome_campanha', 'id_campanha', 'orcamento', 'custo'])
    .agg(
        total_interacoes = ('tipo_interacao', 'count'),
        total_conversoes = ('tipo_interacao', lambda x: (x == 'Conversão').sum())
    )
)

taxa_conversao['taxa_conversao_%'] = (
    (taxa_conversao['total_conversoes'] / taxa_conversao['total_interacoes']) * 100
)

taxa_conversao = taxa_conversao.sort_values('taxa_conversao_%', ascending=False).reset_index()
taxa_conversao


,nome_campanha,id_campanha,orcamento,custo,total_interacoes,total_conversoes,taxa_conversao_%
0,Campanha Nobis,7,1606.62,1595.67,749,90,12.016021
1,Campanha Ipsam,20,9292.03,9555.82,744,89,11.962366
2,Campanha Aspernatur,13,7732.77,5485.33,758,88,11.609499
3,Campanha Velit,1,10168.22,2988.19,699,80,11.444921
4,Campanha Sunt,4,18442.84,6574.57,786,89,11.323155
5,Campanha Libero,8,14551.62,1653.04,778,87,11.182519
6,Campanha Quam,2,14307.26,16227.62,734,80,10.899183
7,Campanha Quisquam,16,3726.32,485.79,736,79,10.733696
8,Campanha Odio,6,19596.79,21774.87,686,73,10.641399
9,Campanha Explicabo,15,15507.16,5054.31,803,82,10.211706


In [26]:
taxa_conversao.to_excel('taxa_conversao.xlsx', index=False)

6. Canais de Marketing:
Analise qual canal de marketing (Email, Google Ads, Meta Ads, etc.) gera maior engajamento dos clientes, baseado nas interações registradas.


In [51]:
canal_interacao = campanhas_marketing.groupby('canal_marketing')['tipo_interacao'].value_counts().sort_values(ascending=False)
canal_interacao



,,count
canal_marketing,tipo_interacao,
Meta Ads,Visualização,4624
Email,Visualização,3235
Google Ads,Visualização,2632
Meta Ads,Clique,1363
Email,Clique,899
Google Ads,Clique,714
Meta Ads,Conversão,679
Email,Conversão,453
Google Ads,Conversão,401


In [28]:
fig = px.bar(canal_interacao, x='canal_marketing', y='count', color='tipo_interacao', text="count")
fig.update_traces(texttemplate='%{text:.0f}', textposition='inside')
fig.update_layout(
    xaxis_title='Canal de Marketing',
    yaxis_title='Quantidade de Interações',
    title='Quantidade de Interações por Canal de Marketing'
)
fig.show()

# C. Análise Integrada (Vendas e Marketing)

7. Relação Temporal:
Investigue se há um aumento nas vendas de determinados produtos logo após o início de uma campanha de marketing.


In [29]:
# Definindo as colunas de datas(object) para datetime
df_campanhas_marketing['data_inicio'] = pd.to_datetime(df_campanhas_marketing['data_inicio'])
df_campanhas_marketing['data_fim'] = pd.to_datetime(df_campanhas_marketing['data_fim'])

In [30]:
#Relacionando as tabelas campanhas com vendas
df_campanhas_vendas = df_campanhas_marketing.join(df_vendas.set_index('id_campanha'), on='id_campanha', how='left')
df_campanhas_vendas

,id_campanha,nome_campanha,canal_marketing,data_inicio,data_fim,orcamento,custo,id_venda,id_cliente,id_produto,data_venda,quantidade,valor_total,canal_aquisicao
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,6,2646,20,2025-07-25,4,2398.08,Inbound
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,9,1743,13,2025-07-07,10,6558.00,Outbound
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,24,3793,12,2025-07-25,6,5386.32,Inbound
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,51,3793,23,2025-07-16,8,2725.28,Inbound
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,77,3866,1,2025-07-12,1,755.55,Outbound
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,580,4430,6,2025-10-25,2,613.36,Inbound
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,622,2671,19,2025-10-18,6,854.10,Outbound
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,632,4181,23,2025-10-28,3,1021.98,Outbound
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,641,4383,17,2025-11-12,10,3116.70,Outbound


In [31]:
#Relaciondo
df_produtos_vendas = df_campanhas_vendas.join(df_produtos.set_index('id_produto'), on='id_produto', how='left')
df_produtos_vendas


,id_campanha,nome_campanha,canal_marketing,data_inicio,data_fim,orcamento,custo,id_venda,id_cliente,id_produto,data_venda,quantidade,valor_total,canal_aquisicao,nome_produto,categoria,preco_unitario,custo_unitario
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,6,2646,20,2025-07-25,4,2398.08,Inbound,Station AI - Harum,SaaS,599.52,311.75
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,9,1743,13,2025-07-07,10,6558.00,Outbound,GS Engage - Hic,SaaS,655.80,537.76
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,24,3793,12,2025-07-25,6,5386.32,Inbound,Demanda Infinita - Expedita,Treinamento,897.72,646.36
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,51,3793,23,2025-07-16,8,2725.28,Inbound,Growth Way - Mollitia,Consultoria,340.66,197.58
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,77,3866,1,2025-07-12,1,755.55,Outbound,Growth Way - Eveniet,Consultoria,755.55,551.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,580,4430,6,2025-10-25,2,613.36,Inbound,High Growth - Quis,Consultoria,306.68,239.21
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,622,2671,19,2025-10-18,6,854.10,Outbound,Station AI - Blanditiis,SaaS,142.35,106.76
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,632,4181,23,2025-10-28,3,1021.98,Outbound,Growth Way - Mollitia,Consultoria,340.66,197.58
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,641,4383,17,2025-11-12,10,3116.70,Outbound,Imersão - Aliquam,Treinamento,311.67,258.69


In [32]:
df_produtos_vendas

,id_campanha,nome_campanha,canal_marketing,data_inicio,data_fim,orcamento,custo,id_venda,id_cliente,id_produto,data_venda,quantidade,valor_total,canal_aquisicao,nome_produto,categoria,preco_unitario,custo_unitario
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,6,2646,20,2025-07-25,4,2398.08,Inbound,Station AI - Harum,SaaS,599.52,311.75
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,9,1743,13,2025-07-07,10,6558.00,Outbound,GS Engage - Hic,SaaS,655.80,537.76
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,24,3793,12,2025-07-25,6,5386.32,Inbound,Demanda Infinita - Expedita,Treinamento,897.72,646.36
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,51,3793,23,2025-07-16,8,2725.28,Inbound,Growth Way - Mollitia,Consultoria,340.66,197.58
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,77,3866,1,2025-07-12,1,755.55,Outbound,Growth Way - Eveniet,Consultoria,755.55,551.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,580,4430,6,2025-10-25,2,613.36,Inbound,High Growth - Quis,Consultoria,306.68,239.21
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,622,2671,19,2025-10-18,6,854.10,Outbound,Station AI - Blanditiis,SaaS,142.35,106.76
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,632,4181,23,2025-10-28,3,1021.98,Outbound,Growth Way - Mollitia,Consultoria,340.66,197.58
19,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,641,4383,17,2025-11-12,10,3116.70,Outbound,Imersão - Aliquam,Treinamento,311.67,258.69


In [33]:
# Filter sales within the campaign period
vendas_durante_campanha = df_produtos_vendas.loc[
    (df_produtos_vendas['data_venda'] >= df_produtos_vendas['data_inicio']) &
    (df_produtos_vendas['data_venda'] <= df_produtos_vendas['data_fim'])
]

vendas_antes_campanha = df_produtos_vendas.loc[
    (df_produtos_vendas['data_venda'] > df_produtos_vendas['data_fim'])
]
vendas_antes_campanha

,id_campanha,nome_campanha,canal_marketing,data_inicio,data_fim,orcamento,custo,id_venda,id_cliente,id_produto,data_venda,quantidade,valor_total,canal_aquisicao,nome_produto,categoria,preco_unitario,custo_unitario


In [34]:

def classificar_periodo(row):
    if row["data_venda"] < row["data_inicio"]:
        return "antes"
    elif row["data_inicio"] <= row["data_venda"] <= row["data_fim"]:
        return "durante"
    else:
        return "depois"

df_produtos_vendas["periodo"] = df_produtos_vendas.apply(classificar_periodo, axis=1)

# Contar vendas por produto e período
resultado = df_produtos_vendas.groupby(["nome_produto", "periodo"]).size().reset_index()
print(resultado)

                   nome_produto  periodo   0
0   Demanda Infinita - Expedita  durante  24
1     Demanda Infinita - Maxime  durante  24
2           GS Engage - Commodi  durante  26
3        GS Engage - Doloremque  durante  30
4               GS Engage - Hic  durante  29
5              Growth Way - Aut  durante  31
6          Growth Way - Eveniet  durante  26
7             Growth Way - Modi  durante  28
8        Growth Way - Molestias  durante  28
9         Growth Way - Mollitia  durante  30
10            High Growth - Est  durante  24
11           High Growth - Nisi  durante  30
12           High Growth - Quis  durante  65
13    High Growth - Repellendus  durante  26
14          High Growth - Rerum  durante  22
15                  Imersão - A  durante  22
16            Imersão - Aliquam  durante  30
17         Imersão - Laboriosam  durante  24
18             Imersão - Maxime  durante  21
19      Station AI - Blanditiis  durante  25
20         Station AI - Dolorem  durante  29
21        

8. Análise Regional:
Explore se determinadas cidades apresentam melhor resposta às campanhas de marketing e como isso se reflete nas vendas.

In [35]:
df_metrica_por_cidade = campanhas_marketing.join(df_clientes.set_index('id_cliente'), on='id_cliente', how='left')


In [36]:
df_vendas_por_cidade = df_campanhas_vendas.join(df_clientes.set_index('id_cliente'), on='id_cliente', how='left')

In [37]:
df_metrica_cidade = df_vendas_por_cidade.join(df_interacoes_marketing.set_index('id_campanha'), on='id_campanha', how='left', lsuffix='_vendas', rsuffix='_interacoes')

In [38]:
agregado_cidade = df_metrica_cidade.groupby('cidade').agg(
    total_interacoes = ('tipo_interacao', 'count'),
    total_conversoes = ('tipo_interacao', lambda x: (x == 'Conversão').sum()),
    total_vendas = ('valor_total', 'sum'),
    total_custo = ('custo', 'sum')
).reset_index()


agregado_cidade['taxa_conversao_%'] = (
    agregado_cidade['total_conversoes'] / agregado_cidade['total_interacoes'] * 100
)


agregado_cidade['custo_por_conversao'] = np.where(
    agregado_cidade['total_conversoes'] > 0,
    agregado_cidade['total_custo'] / agregado_cidade['total_conversoes'],
    np.nan
)

agregado_cidade['roas'] = np.where(
    agregado_cidade['total_custo'] > 0,
    agregado_cidade['total_vendas'] / agregado_cidade['total_custo'],
    np.nan
)



agregado_cidade = agregado_cidade.head(5).sort_values(ascending=False, by='taxa_conversao_%').round(2)


In [39]:
agregado_cidade

,cidade,total_interacoes,total_conversoes,total_vendas,total_custo,taxa_conversao_%,custo_por_conversao,roas
1,Albuquerque,1564,176,3198475.22,6453677.14,11.25,36668.62,0.50
2,Almeida,2216,247,3882427.36,7824612.96,11.15,31678.59,0.50
4,Almeida do Galho,734,80,395207.62,11911073.08,10.90,148888.41,0.03
3,Almeida de Silveira,736,79,902865.92,357541.44,10.73,4525.84,2.53
0,Abreu,725,69,22431.50,7542726.00,9.52,109314.87,0.00


In [40]:
fig = px.pie(
    agregado_cidade,
    names='cidade',
    values=agregado_cidade['taxa_conversao_%'],
    title='Taxa de Conversão por Cidade'
)
fig.update_traces(textinfo='percent+label')
fig.show()

In [41]:
fig = px.bar(agregado_cidade, x='cidade', y='total_vendas', text='total_vendas')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    xaxis_title='Cidade')
fig.show()

In [42]:
df_principal = df_vendas.merge(df_produtos, on='id_produto', how='left')

In [43]:
df_principal = df_campanhas_marketing.merge(df_principal, on='id_campanha', how='left')

In [44]:
df_principal = df_principal.merge(df_clientes, on='id_cliente', how='left')

In [45]:
df_principal = df_principal.merge(df_interacoes_marketing, on='id_campanha', how='left')

In [46]:
df_principal

,id_campanha,nome_campanha,canal_marketing,data_inicio,data_fim,orcamento,custo,id_venda,id_cliente_x,id_produto,...,categoria,preco_unitario,custo_unitario,nome,cidade,segmento,id_interacao,id_cliente_y,data_interacao,tipo_interacao
0,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,6,2646,20,...,SaaS,599.52,311.75,Maria Fernanda Cardoso,Rezende,B2B,14,3189,2025-06-19,Visualização
1,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,6,2646,20,...,SaaS,599.52,311.75,Maria Fernanda Cardoso,Rezende,B2B,51,1758,2025-04-21,Visualização
2,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,6,2646,20,...,SaaS,599.52,311.75,Maria Fernanda Cardoso,Rezende,B2B,64,206,2025-01-21,Visualização
3,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,6,2646,20,...,SaaS,599.52,311.75,Maria Fernanda Cardoso,Rezende,B2B,72,4202,2025-05-24,Visualização
4,1,Velit,Google Ads,2025-07-05,2025-07-30,10168.22,2988.19,6,2646,20,...,SaaS,599.52,311.75,Maria Fernanda Cardoso,Rezende,B2B,80,4915,2025-07-09,Conversão
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493469,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,642,4780,16,...,Consultoria,467.21,247.62,Dra. Laura Nunes,Rocha,B2C,14811,2732,2025-05-19,Conversão
493470,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,642,4780,16,...,Consultoria,467.21,247.62,Dra. Laura Nunes,Rocha,B2C,14842,1990,2025-09-25,Visualização
493471,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,642,4780,16,...,Consultoria,467.21,247.62,Dra. Laura Nunes,Rocha,B2C,14892,4426,2025-03-05,Clique
493472,20,Ipsam,Meta Ads,2025-10-18,2025-11-16,9292.03,9555.82,642,4780,16,...,Consultoria,467.21,247.62,Dra. Laura Nunes,Rocha,B2C,14936,2488,2025-08-15,Visualização


In [47]:
df_principal.to_excel('df_principal.xlsx', index=False)